In [1]:
import pandas as pd
import numpy as np
#import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import time

In [3]:
%%time 

# Abriendo los archivos .csv
folderpath =  r"data"
print("Files and directories in a specified path:")
files = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
data\endes_2021_programas_sociales_hogar.csv
data\endes_2021_RE223132_nacimiento.csv
data\endes_2021_RE516171_nupcialidad_fecundidad.csv
data\endes_2021_RE758081_sida_condon.csv
data\endes_2021_REC84DV_mortalidad_violencia.csv
data\endes_2021_REC91_MEF.csv
data\endes_2021_REC93DV_disciplina_infantil.csv
data\endes_2021_REC95_inmunizacion_salud.csv
data\endes_2021_RECH23_vivienda.csv
Wall time: 3.99 ms


In [ ]:
# Uniendo los files
#data = pd.concat(map(pd.read_csv, files), ignore_index=True)

### Import csv using dask

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

In [5]:
client

Client Scheduler: tcp://127.0.0.1:53052 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 12.68 GB


In [8]:
import dask.dataframe as dd

C:\Users\Yoseph\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\Yoseph\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\Yoseph\anaconda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [7]:
 files

['data\\endes_2021_programas_sociales_hogar.csv',
 'data\\endes_2021_RE223132_nacimiento.csv',
 'data\\endes_2021_RE516171_nupcialidad_fecundidad.csv',
 'data\\endes_2021_RE758081_sida_condon.csv',
 'data\\endes_2021_REC84DV_mortalidad_violencia.csv',
 'data\\endes_2021_REC91_MEF.csv',
 'data\\endes_2021_REC93DV_disciplina_infantil.csv',
 'data\\endes_2021_REC95_inmunizacion_salud.csv',
 'data\\endes_2021_RECH23_vivienda.csv']

In [35]:
#guardamos los dask dataframes en una lista
dfs = []

for file in files:
    
    df = dd.read_csv(file, dtype=str)
    dfs.append(df)

In [36]:
#asignamos nombres a los dataframes
programas_sociales = dfs[0]
nacimiento = dfs[1]
nupcialidad_fecundidad = dfs[2]
sida_condon = dfs[3]
mortalidad_violencia = dfs[4]
MEF = dfs[5] 
disciplina_infantil = dfs[6]
inmunizacion_salud = dfs[7]
vivienda = dfs[8]

In [37]:
#identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud


### Merging dfs

In [42]:
df_merged = nacimiento.merge(nupcialidad_fecundidad,on=['CASEID', 'ID1'], how='left' ).merge(sida_condon,on=['CASEID', 'ID1'], how='left' ).merge(mortalidad_violencia,on=['CASEID', 'ID1'], how='left' ).merge(MEF,on=['CASEID', 'ID1'], how='left' ).merge(disciplina_infantil,on=['CASEID', 'ID1'], how='left' ).merge(inmunizacion_salud,on=['CASEID', 'ID1'], how='left' )

C:\Users\Yoseph\anaconda3\lib\site-packages\dask\dataframe\multi.py:340: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ID1_x'} in the result is deprecated and will raise a MergeError in a future version.
  meta = left._meta_nonempty.merge(right._meta_nonempty, **kwargs)
C:\Users\Yoseph\anaconda3\lib\site-packages\dask\dataframe\multi.py:340: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ID1_x'} in the result is deprecated and will raise a MergeError in a future version.
  meta = left._meta_nonempty.merge(right._meta_nonempty, **kwargs)


In [51]:
df_final = df_merged.compute()

In [71]:
df_final

,ID1_x,CASEID,V201,V202,V203,V204,V205,V206,V207,V208,...,S466MC11,S466YC11,S466WC11,S466HC11,S466DC12,S466MC12,S466YC12,S466WC12,S466HC12,violencia_sum
0,2021,000100301 2,1,1,0,0,0,0,0,1,...,,,,,,,,,,0000
1,2021,000100301 4,1,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,2021,000100401 1,6,4,0,0,2,0,0,1,...,3,2020,8640,725,26,5,2020,9010,750,1100
3,2021,000100801 2,4,1,3,0,0,0,0,1,...,,,,,,,,,,
4,2021,000100801 4,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43132,2021,325410601 1,7,4,3,0,0,0,0,2,...,,,,,,,,,,0000
43133,2021,325410601 1,7,4,3,0,0,0,0,2,...,7,2019,9950,790,29,10,2019,11300,860,0000
43134,2021,325410601 2,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
43135,2021,325410801 2,2,1,1,0,0,0,0,1,...,,,,,,,,,,0000


In [64]:
df_final.dtypes

ID1_x            object
CASEID           object
V201             object
V202             object
V203             object
                  ...  
S466MC12         object
S466YC12         object
S466WC12         object
S466HC12         object
violencia_sum    object
Length: 1279, dtype: object

In [ ]:
for column in df_final: 
    
    print(column)
    
    try:
        if df_final[column].dtypes == 'O':
        
            df_final[column] = df_final[column].str.replace(" ", "")
            # Reemplazando los missing values por NaN
            df_final[column] = df_final[column].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            df_final[column] = pd.to_numeric(df_final[column], errors='coerce')

            # Convirtiendo los datos a valores Int
            df_final[column] = df_final[column].astype('Int64') 
    except:
        print(f"{column}: did not work")

ID1_x
ID1_x: did not work
CASEID
V201
V202
V203
V204
V205
V206
V207
V208
V209
V210
V211
V212
V213
V214
V215
V216
V217
V218
V219
V220
V221
V222
V223
V224
V225
V226
V227
V228
V229
V230
V231
V232
V233
V234
V235
V237
V238
V239
V240
V241
V242
V243
V301
V302
V310
V311
V312
V313
V315
V316
V317
V318
V319
V320
V321
V322
V323
V323A
V325A
V326
V327
V337
V359
V360
V361
V362
V363
V364
V367
V372
V372A
V375A
V376
V376A
V379
V380
V384A
V384B
V384C
V393
V394
V395
V3A01
V3A02
V3A03
V3A04
V3A05
V3A06
V3A07
V3A08A
V3A08B
V3A08C
V3A08D
V3A08E
V3A08F
V3A08G
V3A08H
V3A08I
V3A08J
V3A08K
V3A08L
V3A08M
V3A08N
V3A08O
V3A08P
V3A08Q
V3A08R
V3A08S
V3A08T
V3A08U
V3A08V
V3A08W
V3A08X
V3A08Z
V3A09A
V3A09B
QI302_05A
QI302_05B
V305_01
V305_02
V305_03
V305_05
V305_06
V305_07
V305_08
V305_09
V305_10
V305_11
V305_13
V305_14
V305_15
V305_16
V307_01
V307_02
V307_03
V307_04
V307_05
V307_06
V307_07
V307_08
V307_09
V307_10
V307_11
V307_12
V307_13
V307_14
V307_15
V307_16
ID1_y
ID1_y: did not work
V501
V502
V503
V504
V505
V506
V5

In [56]:
df_final.dtypes

ID1_x            object
CASEID           object
V201             object
V202             object
V203             object
                  ...  
S466MC12         object
S466YC12         object
S466WC12         object
S466HC12         object
violencia_sum    object
Length: 1279, dtype: object

### Creación de por lo menos dos variables 

*Las variables de interés son las siguientes:*

D104: Hubo violencia emocional | 0=No, 1=Sí

D106: Experimentó cualquier violencia física menos severa | 0=No, 1=Sí

D107: Experimentó cualquier violencia física severa | 0=No, 1=Sí

D108: Experimentó cualquier violencia sexual | 0=No, 1=Sí

In [50]:
df_merged.D104.value_counts().compute()

0    21544
     14499
1     6259
Name: D104, dtype: int64

In [47]:
df_merged['violencia_sum'] = df_merged.D104 + df_merged.D106 + df_merged.D107 + df_merged.D108

In [49]:
df_merged['violencia_sum'].value_counts().compute()

0000    18395
        14499
0100     2434
1100     1979
1000     1822
1110     1132
1111      817
0110      432
1101      403
0101      100
0001       86
1001       84
0111       61
0010       30
1010       20
0011        6
1011        2
Name: violencia_sum, dtype: int64

In [12]:
mortalidad_violencia['violencia_sum'].value_counts().compute()

0000    14362
        14112
0100     1866
1100     1611
1000     1425
1110      901
1111      679
0110      337
1101      324
0101       74
1001       66
0001       57
0111       52
0010       22
1010       16
0011        3
1011        2
Name: violencia_sum, dtype: int64

#### Explorando las variables

In [46]:
mortalidad_violencia.D104.value_counts().compute()

0    16773
     14112
1     5024
Name: D104, dtype: int64

In [47]:
mortalidad_violencia.D106.value_counts().compute()

0    15953
     14112
1     5844
Name: D106, dtype: int64

In [48]:
mortalidad_violencia.D107.value_counts().compute()

0    19785
     14112
1     2012
Name: D107, dtype: int64

In [49]:
mortalidad_violencia.D108.value_counts().compute()

0    20540
     14112
1     1257
Name: D108, dtype: int64

Creamos una variable que tome el valor de 1 si es que la mujer ha sido víctima de violencia física y 0 si es que no lo ha sido

In [51]:
import dask.array as da

In [74]:
mortalidad_violencia["V_FISICA"] = (mortalidad_violencia["D106"].fillna(0) == 1) | (mortalidad_violencia["D107"].fillna(0) == 1)


In [69]:
mortalidad_violencia["V_FISICA"] = da.where( 
                                    ( (mortalidad_violencia["D106"] == 1) | (mortalidad_violencia["D107"] == 1) ).values
                                    , 1, 0
                                    )

In [75]:
mortalidad_violencia["V_FISICA"].value_counts().compute()

False    35909
Name: V_FISICA, dtype: int64

In [7]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'HHID', 'HV202', 'HV205', 'HV206', 'HV247',   # serv. bas.
             'HV221', 'HV216',                             # serv. bas.
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = data[variables]
data_final.shape

Wall time: 1min 51s


(293282, 34)

In [93]:
data_final['violencia'] = (data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108'])

C:\Users\sandr\anaconda3\envs\renv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [117]:
data_final['violencia'].value_counts()

0000    14362
        14112
0100     1866
1100     1611
1000     1425
1110      901
1111      679
0110      337
1101      324
0101       74
1001       66
0001       57
0111       52
0010       22
1010       16
0011        3
1011        2
Name: violencia, dtype: int64

In [144]:
data_final['D104'].dropna()
data_final['D106'].dropna()

141594    0
141596    1
141599    0
141600    0
141601    0
         ..
177497    0
177498    0
177499    0
177501    0
177502    0
Name: D106, Length: 21797, dtype: Int64

In [146]:
data_final['D104'] = pd.to_numeric(data_final['D104'], errors='coerce').astype('int32')
data_final['D106'] = pd.to_numeric(data_final['D104'], errors='coerce').astype('int32')

ValueError: cannot convert to 'int32'-dtype NumPy array with missing values. Specify an appropriate 'na_value' for this dtype.

In [145]:
data_final.eval('violencia = D104 + D106')

TypeError: Cannot interpret 'Int64Dtype()' as a data type

In [140]:
data_final['D106'].value_counts()
# data_final.astype({'D104':'str'}).dtypes()

0    16773
1     5024
Name: D106, dtype: Int64

In [104]:
data_final['violencia'] = data_final['violencia'].dropna()

C:\Users\sandr\anaconda3\envs\renv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
if data_final['violencia'] == '0000':
    data_final['violencia'] == 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [13]:
data_final['violencia'] == (data_final['violencia'] == '0000') == 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [63]:
if data_final['violencia'].all() == 1: 
    data_final['violencia'] == 1

In [74]:
data_final['violencia'].dropna()

141594    0000
141595        
141596    1100
141597        
141598        
          ... 
177498    0000
177499    0000
177500        
177501    0000
177502    0000
Name: violencia, Length: 35909, dtype: object

In [77]:
data_final['violencia']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
293277    NaN
293278    NaN
293279    NaN
293280    NaN
293281    NaN
Name: violencia, Length: 293282, dtype: object

In [ ]:
%%time 

data_final.eval('violencia = D104 + D106')
# 
=  = (data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108'])

In [240]:
data_final['violencia'].value_counts()

0000    14362
        14112
0100     1866
1100     1611
1000     1425
1110      901
1111      679
0110      337
1101      324
0101       74
1001       66
0001       57
0111       52
0010       22
1010       16
0011        3
1011        2
Name: violencia, dtype: int64

In [245]:
data_final['violencia']

17

In [ ]:
data_final.sum()

In [ ]:
.loc[(daskdd["ocupinf"].isin([1,2]))].replace({'ocupinf': 2},0)

In [241]:
data_final.loc[(data_final['violencia'].isin([0000,1011]))].replace({'violencia': 0000},0)

,ID1,SREGION,SPROVIN,SDISTRI,CASEID,HHID,HV202,HV205,HV206,HV247,...,S45B3,S45F1,S45F2,V501,V714,D104,D106,D107,D108,violencia


In [237]:
data_final['D104'].value_counts()

0    16773
     14112
1     5024
Name: D104, dtype: int64

In [211]:
data_final['D106'].value_counts()

0    15953
     14112
1     5844
Name: D106, dtype: int64

In [212]:
data_final['D107'].value_counts()

0    19785
     14112
1     2012
Name: D107, dtype: int64

In [213]:
data_final['D108'].value_counts()

0    20540
     14112
1     1257
Name: D108, dtype: int64